# RA2CE feature: Single link losses analysis

## Single link losses Analysis

In [ ]:
from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler
import geopandas as gpd
import pyproj

# Import config data
root_dir = Path("data", "single_link_losses")
assert root_dir.exists()

# Load network data.
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
_network_config_data = NetworkConfigDataReader().read(root_dir.joinpath("network.ini"))

# Load analysis data.
from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader
_analysis_config_data = AnalysisConfigDataReader().read(root_dir.joinpath("analysis.ini"))

### Modify the analysis config data with custom properties

In [2]:
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisSectionLosses

# Modify the first analysis (assuming it's the only one)
_found_analysis: AnalysisSectionLosses = _analysis_config_data.analyses[0]
_input_analysis_path = root_dir.joinpath("input_analysis_data")
_found_analysis.traffic_intensities_file = _input_analysis_path.joinpath("traffic_intensities.csv")
_found_analysis.resilience_curves_file = _input_analysis_path.joinpath("resilience_curve.csv")
_found_analysis.values_of_time_file = _input_analysis_path.joinpath("values_of_time.csv")

### Configure network

Let's have a look first at the hazard overlay to locate the road segments for which losses will be expected. We run only the network configuration.

In [3]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)
handler.configure()

In [ ]:
hazard_output = root_dir / "static" / "output_graph" / "base_graph_hazard_edges.gpkg"
hazard_gdf = gpd.read_file(hazard_output, driver = "GPKG")
hazard_gdf.crs = pyproj.CRS(4326)
hazard_gdf.explore(column="EV1_ma", cmap = "viridis", tiles="CartoDB positron")

### Run losses analysis

In [ ]:
handler.run_analysis()

Let's inspect the results of the analysis. For every segment in the network, the losses are expressed in vehicle loss hour (vlh) for all the types of trip defined in the input files: business, commute, freight and other.

In [ ]:
analysis_output_folder = root_dir / "output" / "single_link_losses" # specify path to output folder

losses_gdf = gpd.read_file(analysis_output_folder/"beira_single_losses.gpkg") #specify the name of the geopackage holding your results (can be found in the analysis output folder)
losses_gdf.head() #display the attributes of the file

The map below displays the total loss for all trip types per link

In [ ]:
losses_roads_map = losses_gdf.explore(column='vlh_EV1_ma_total', tiles="CartoDB positron",)
display(losses_roads_map)